In [2]:
Ticker = 'NVDA'
stock = yf.Ticker(Ticker)
earnings = stock.get_earnings_dates()
earnings

,EPS Estimate,Reported EPS,Surprise(%)
Earnings Date,,,
2025-05-20 16:00:00-04:00,NaN,NaN,NaN
2025-02-19 16:00:00-05:00,NaN,NaN,NaN
2024-11-19 16:00:00-05:00,NaN,NaN,NaN
2024-08-28 16:00:00-04:00,0.64,NaN,NaN
2024-05-22 16:00:00-04:00,0.56,0.61,0.0948
2024-02-21 16:00:00-05:00,4.64,5.16,0.1132
2023-11-21 16:00:00-05:00,3.37,4.02,0.1921
2023-08-23 16:00:00-04:00,2.09,2.70,0.2941
2023-05-24 16:00:00-04:00,0.92,1.09,0.1857


In [146]:

data_new = yf.Ticker(Ticker).history(period = '31mo', interval ='1wk', auto_adjust = False)
data_old = yf.Ticker(Ticker).history(period = '62mo', interval ='1wk', auto_adjust = False)
def Prep(df, old):
    df.drop('Close', axis = 1, inplace = True)
    df['Dividends'] = df['Dividends']/df['Adj Close']
    df['dir'] = (df['Adj Close'].shift(-3) - df['Adj Close'])
    if old == True:
        df = df.loc[~df.index.isin(data_new.index)]
    df.loc[df['dir'] > 0, 'dir'] = 1
    df.loc[df['dir'] < 0, 'dir'] = 0
    if old == False:
        df['dir'] = df['dir'].fillna('insf')
    return df
df_new = Prep(data_new, old = False).copy()
df_old = Prep(data_old, old = True).copy()
def Cal(df):
    df['std'] = df['Adj Close'].rolling(window = 12, min_periods = 12).std()
    
    dof = np.sign(df['Adj Close'] - df['Adj Close'].shift(1))
    Vol_Change = (df['Volume'] * dof)
    bov = Vol_Change.cumsum()
    df['BOV'] = bov.pct_change()
#Fibonnachi sequence
    Low_60 = df['Adj Close'].rolling(window = 60, min_periods = 60).min()
    High_60 = df['Adj Close'].rolling(window = 60, min_periods = 60).max()
    Dif = High_60 - Low_60
    def Low_cal(k):
        arr = Low_60 + (Dif * k)
        Parr = (df['Adj Close']/arr) -1
        return Parr
    def High_cal(j):
        arr = High_60 - (Dif * j)
        Parr = (df['Adj Close']/arr) -1
        return Parr
    df['SF1'] = Low_cal(0.382)
    df['SF2'] = Low_cal(0.5)
    df['SF3'] = Low_cal(0.618)
    df['SF4'] = Low_cal(0.786)
    df['RC1'] = High_cal(0.786)
    
#Boolinger Bands       
    SMA = df['Adj Close'].rolling(window = 40, min_periods = 40).mean()
    STD = df['Adj Close'].rolling(window = 40, min_periods = 40).std()
    LBol = SMA - (2* STD)
    HBol = SMA + (2* STD)
    df['PLbol'] = ((df['Adj Close'] - LBol) / LBol) 
    df['PHbol'] = ((df['Adj Close'] - HBol) / HBol)
    
#Ichimoku Cloud    
    high_prices = df['High']
    close_prices = df['Adj Close']
    low_prices = df['Low']
    nine_period_high =  df['High'].rolling(window=9).max()
    nine_period_low = df['Low'].rolling(window=9).min()
    tenkan_sen = (nine_period_high + nine_period_low) /2
    period26_high = high_prices.rolling(window=26).max()
    period26_low = low_prices.rolling(window=26).min()
    kijun_sen = (period26_high + period26_low) / 2
    senkou_span_a = ((tenkan_sen + kijun_sen) / 2).shift(26)
    period52_high = high_prices.rolling(window=52).max()
    period52_low = low_prices.rolling(window=52).min()
    senkou_span_b = ((period52_high + period52_low) / 2).shift(26)
    df['kumo'] = senkou_span_a - senkou_span_b
    df['kijun_tenkan'] = tenkan_sen - kijun_sen

#MACD
    ema12 = df['Adj Close'].ewm(span = 13,adjust=False, min_periods = 12).mean()
    ema26 = df['Adj Close'].ewm(span = 26,adjust=False, min_periods = 26).mean()
    MACD = ema12 - ema26
    df['MACD'] = MACD
    ema9 = df['MACD'].ewm(span = 9,adjust=False, min_periods = 9).mean()
    df['ema9'] = ema9
    df['MACD_grad'] = df['MACD'] - df['MACD'].shift(1)
    df['ema9_grad'] = df['ema9'] - df['ema9'].shift(1)
    df['MACD_max'] = df['MACD'].rolling(window = 84).max()
    df['ema9_max'] = df['ema9'].rolling(window = 84).max()
    
#RSI
    cup = df['Adj Close'].diff().copy()
    cdown = df['Adj Close'].diff().copy()
    cup[cup < 0] = 0
    cdown[cdown > 0] = 0
    cup_sum = cup.rolling(14, min_periods = 14).sum()
    cdown_sum = cdown.rolling(14, min_periods = 14).sum() 
    rsi_cal = cup_sum / (cdown_sum * -1)
    RSI = 100 - (100/(1 + rsi_cal))
    df['RSI'] = RSI

#ATR
    def wwma(values, n):
        return values.ewm(alpha=1/n, adjust=False).mean()
    def atr(df, n=14):
        data_work = df.copy()
        high = data_work['High']
        low = data_work['Low']
        close = data_work['Adj Close']
        data_work['tr0'] = abs(high - low)
        data_work['tr1'] = abs(high - close.shift())
        data_work['tr2'] = abs(low - close.shift())
        tr = data_work[['tr0', 'tr1', 'tr2']].max(axis=1)
        atr = wwma(tr, n)
        return atr
    df['atr'] = atr(df, 14)
    
    return df

df_com = pd.concat([Cal(df_old), Cal(df_new)])
df_com.index = df_com.index.strftime('%Y-%m-%d')


In [5]:

stock = yf.Ticker('NVDA')
print(stock.earnings_dates['EPS Estimate'].dropna()[0])
print(stock.earnings_dates['Reported EPS'].dropna()[0])
print((stock.earnings_dates['Surprise(%)'].dropna() > 0).mean())

5.51
5.16
0.75


In [7]:
luist = []
import yfinance as yf
import pandas as pd
import numpy as np
import time as time
from datetime import datetime
stock_ticks = ['ABBV', 'TJX', 'AMD', 'GE','RSG','KO', 'V', 'PG', 'CL', 'L', 'MMC', 'ROP', 'MCD', 'TMUS', 'WM', 'MA', 'COR', 
                 'YUM', 'PM', 'TJX', 'JNJ', 'KMB', 'HON', 'ITW', 'ATO', 'PEP', 'FI', 'ICE', 'AME', 
                 'LIN', 'KMI', 'WMT', 'JPM', 'LMT', 'MDLZ', 'GD', 'OTIS', 'CHD', 'DUK', 'HIG', 'MO',
                 'CSX', 'AJG', 'WMB', 'STZ', 'CME', 'APH', 'MRK', 'SYY', 'MSI', 'ED', 'AMP', 'ABT', 
                 'VRSK', 'AIG', 'COST', 'VRSN', 'ECL', 'BRO', 'SO', 'PEG', 'LH', 'PPL', 'ABBV', 'CB', 
                 'HOLX', 'BR', 'AVY', 'CTAS', 'IEX', 'MCK', 'KDP', 'TRV', 'DRI', 'CNP', 'ADP', 'FE',
                 'CSCO', 'KHC', 'EA', 'PAYX', 'SRE', 'MDT', 'CMS', 'PRU', 'GIS', 'OKE', 'CBOE', 'LNT',
                 'HD', 'ETR', 'DGX', 'AFL', 'WEC', 'BK', 'JKHY', 'LYB', 'SPGI', 'HLT', 'XYL', 'DOV', 
                 'EXPD', 'VICI', 'PPG', 'ORLY', 'SHW', 'BSX', 'NI', 'LDOS', 'DTE', 'MCO', 'BDX', 'O',
                 'BLK', 'REGN', 'WRB', 'DOW', 'UNP', 'PFG', 'FDS', 'PTC', 'CPRT', 'EVRG', 'TEL', 'CVX', 
                 'HSY', 'AON', 'AVB', 'AAPL', 'FAST', 'INVH', 'MET', 'KR', 'PNW', 'WAB', 'AEP', 'REG', 
                 'TDG', 'VMC', 'K', 'CAG', 'NDSN', 'CTSH', 'IBM', 'SYK', 'AEE', 'PCG', 'FTV', 'XOM',
                 'FFIV', 'EIX', 'EQR', 'GWW', 'WELL', 'ROST', 'MSFT', 'EXC', 'PKG', 'AZO', 'J', 'TMO', 
                 'PCAR', 'NOC', 'ZBH', 'ACN', 'CINF', 'MNST', 'MAR', 'STE', 'LOW', 'AXP', 'CAH', 'HSIC',
                 'RJF', 'LHX', 'BMY', 'GS', 'FRT', 'CDW', 'ELV', 'WTW', 'NWSA', 'C', 'AOS', 'NWS', 'SJM',
                 'TRGP', 'WY', 'GLW', 'IR', 'SNA', 'MLM', 'EMR', 'CMCSA', 'GILD', 'FOX', 'TAP', 'AWK', 
                 'AMCR', 'TSN', 'SPG', 'WFC', 'MAA', 'XEL', 'RTX', 'COP', 'UNH', 'OMC', 'ACGL', 'DHR', 
                 'FOXA', 'CMG', 'BG', 'CMI', 'BKR', 'TSCO', 'TDY', 'HCA', 'AIZ', 'OXY', 'ALL', 'FANG', 
                 'CPB', 'UDR', 'AKAM', 'IRM', 'ESS', 'NDAQ', 'PSA', 'HII', 'ROL', 'TTWO', 'RHI', 'PNR',
                 'CTRA', 'EG', 'ALLE', 'NSC', 'DE', 'VZ', 'EMN', 'BAC', 'VRTX', 'UPS', 'UHS', 'CLX', 
                 'STT', 'TXN', 'T', 'PH', 'EOG', 'CPT', 'COO', 'ETN', 'NVR', 'BIIB', 'HST', 'TYL', 'MS',
                 'TXT', 'MKC', 'FIS', 'PFE', 'GRMN', 'PSX', 'CPAY', 'INCY', 'AMGN', 'TT', 'FDX', 'JCI',
                 'KVUE', 'GPC', 'LKQ', 'COF', 'MAS', 'MOH', 'D', 'CNC', 'AMT', 'EQIX', 'IT', 'GE', 'HES',
                 'CDNS', 'PNC', 'KIM', 'TROW', 'PLD', 'CCI', 'JBHT', 'INTU', 'PGR', 'HUBB', 'VTR', 'SBUX',
                 'DPZ', 'BKNG', 'DVN', 'VTRS', 'ES', 'IPG', 'HWM', 'SLB', 'DD', 'DLR', 'EBAY', 'VLTO', 
                 'ZTS', 'ADSK', 'IQV', 'DIS', 'NKE', 'NRG', 'MPC', 'KEYS', 'ISRG', 'CBRE', 'ADI', 'SNPS',
                 'NUE', 'HAL', 'CE', 'A', 'GPN', 'SYF', 'WRK', 'BAX', 'VLO', 'CAT', 'HBAN', 'APD', 'LYV', 
                 'GOOG', 'TFX', 'AMZN', 'CI', 'HRL', 'GOOGL', 'BEN', 'NTRS', 'MSCI', 'WYNN', 'NEE', 'EW', 
                 'PWR', 'BALL', 'CARR', 'POOL', 'STLD', 'CF', 'DOC', 'HPQ', 'IP', 'CHRW', 'MMM', 'GM', 'DAL', 
                 'LVS', 'MRO', 'IDXX', 'SBAC', 'BBY', 'CSGP', 'JNPR', 'ULTA', 'SCHW', 'TRMB', 'MTB', 'NTAP',
                 'FICO', 'BIO', 'WAT', 'EFX', 'LLY', 'ANSS', 'RL', 'WST', 'LEN', 'NOW', 'ROK', 'GEHC', 'HPE',
                 'QCOM', 'NXPI', 'DLTR', 'PHM', 'CTVA', 'SWK', 'TFC', 'BX', 'BA', 'DHI', 'RF', 'FITB', 'CVS',
                 'USB', 'SWKS', 'RVTY', 'EXR', 'CRL', 'MGM', 'ODFL', 'MTD', 'STX', 'RCL', 'ADBE', 'EQT', 'IVZ',
                 'APA', 'LW', 'BWA', 'KLAC', 'MCHP', 'ORCL', 'AXON', 'TGT', 'GEN', 'HUM', 'MHK', 'QRVO', 'TPR',
                 'AMAT', 'F', 'ADM', 'LRCX', 'MOS', 'HAS', 'IFF', 'ARE', 'VST', 'CRM', 'LULU', 'WDC', 'ABNB', 
                 'FCX', 'TER', 'DAY', 'APTV', 'UBER', 'AVGO', 'CEG', 'CFG', 'NEM', 'CHTR', 'DG', 'TECH', 'NFLX',
                 'DFS', 'DVA', 'PYPL', 'LUV', 'META', 'AES', 'URI', 'MKTX', 'MU', 'MTCH', 'SOLV', 'BBWI', 'KMX',
                 'DECK', 'WBA', 'CMA', 'AAL', 'UAL', 'KEY', 'INTC', 'RMD', 'CTLT', 'ILMN', 'ZBRA', 'DXCM', 'BXP',
                 'FTNT', 'JBL', 'MPWR', 'CZR', 'ANET', 'EL', 'NVDA', 'PODD', 'EXPE', 'ON', 'BLDR', 'FMC', 'ALGN',
                 'CCL', 'AMD', 'ETSY', 'EPAM', 'FSLR', 'PANW', 'GNRC', 'WBD', 'TSLA', 'NCLH', 'ALB', 'PAYC', 'GEV',
                 'MRNA', 'PARA', 'ENPH', 'GL', 'SMCI', 'BRK.B', 'BF.B']


In [8]:
def ccal(df):
    try:
        yoyIncrease = df['EPS Estimate'].dropna()[0] > df.dropna(subset = ['Reported EPS'])['Reported EPS'][3]
        supriseConsi = ((df['Surprise(%)'].dropna() > 0).mean()) == 1 
        increaseEarnings = (df['EPS Estimate'].dropna()[0] > df['Reported EPS'].dropna()[0])
        conditions = yoyIncrease and supriseConsi and increaseEarnings       
        
        peakEarnings = data['Reported EPS'].dropna()[1:4].max() <= data['EPS Estimate'].dropna()[0]
    except:
        print(f'{ticker}, Failed')
        return 0
    if conditions:
        if peakEarnings:
            return 1, 1
        else:
            return 1, 0
    else:
        return 0, 0


In [9]:


for ticker in stock_ticks:
    
    try:
        stock = yf.Ticker(ticker)
        data = stock.earnings_dates
        rating = ccal(data)
        if rating == (1, 1):
            luist.append([ticker, data[data['Reported EPS'].isna()].index[-1].date(), True])
        elif rating == (1, 0):
            luist.append([ticker, data[data['Reported EPS'].isna()].index[-1].date(), False])
    except KeyError:
        print(f'{ticker}, Failed')





L, Failed
SO, Failed
PEG, Failed
LH, Failed
PPL, Failed
CB, Failed
HOLX, Failed
BR, Failed
AON, Failed
AVB, Failed
AAPL, Failed
FAST, Failed
INVH, Failed
MET, Failed
KR, Failed
PNW, Failed
WAB, Failed
AEP, Failed
REG, Failed
TDG, Failed
NWS, Failed
FOX, Failed
TSN, Failed
SPG, Failed
WFC, Failed
MAA, Failed
XEL, Failed
RTX, Failed
COP, Failed
UNH, Failed
OMC, Failed
ACGL, Failed
DHR, Failed
CPAY, Failed
JCI, Failed
KVUE, Failed
GPC, Failed
LKQ, Failed
COF, Failed
MAS, Failed
MOH, Failed
D, Failed
CNC, Failed
AMT, Failed
EQIX, Failed
IT, Failed
DD, Failed
EW, Failed
PWR, Failed
BALL, Failed
CARR, Failed
POOL, Failed
STLD, Failed
CF, Failed
DOC, Failed
HPQ, Failed
IP, Failed
CHRW, Failed
BWA, Failed
KLAC, Failed
MCHP, Failed
ORCL, Failed
AXON, Failed
TGT, Failed
GEN, Failed
HUM, Failed
MHK, Failed
QRVO, Failed
TPR, Failed
AMAT, Failed
DAY, Failed
SOLV, Failed
FMC, Failed
ALGN, Failed
CCL, Failed
AMD, Failed
ETSY, Failed
EPAM, Failed
FSLR, Failed
PANW, Failed
GNRC, Failed
WBD, Failed
TSLA

BRK.B: $BRK.B: possibly delisted; No earnings dates found


BRK.B, Failed


BF.B: $BF.B: possibly delisted; No earnings dates found


BF.B, Failed


In [17]:
data = yf.Ticker('GEV')  
data = data.earnings_dates
ccal(data)

BF.B, Failed


0

In [23]:
output = pd.DataFrame(luist, columns = ['tickers', 'next_earnings', 'Peak']).sort_values(['Peak', 'next_earnings'], ascending = [False, True])
k = [i for i in (output['tickers'])]
k

['ADBE',
 'AZO',
 'CTAS',
 'PEP',
 'PG',
 'ABT',
 'SLB',
 'CDNS',
 'V',
 'FI',
 'MSFT',
 'ROP',
 'NOW',
 'RSG',
 'MA',
 'LIN',
 'RCL',
 'MPWR',
 'XYL',
 'ALLE',
 'ETN',
 'GPN',
 'MSCI',
 'TEL',
 'BKNG',
 'CI',
 'SNPS',
 'ROST',
 'LULU',
 'ADP',
 'HON',
 'DECK',
 'IQV',
 'QCOM',
 'MSI',
 'ED']

In [24]:
output_path = r'C:\Users\Jerome\Desktop\Jerome_Ground\Stocker_Git\AI_Earning_Scraper\08-10-24.csv'

output.to_csv(output_path, index = False)

                                              0.0/73.5 kB ? eta -:--:--
     ---------------------------------------- 73.5/73.5 kB ? eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.37
    Uninstalling yfinance-0.2.37:
      Successfully uninstalled yfinance-0.2.37
Note: you may need to restart the kernel to use updated packages.


In [14]:
Ticker = 'PAYC'
stock = yf.Ticker(Ticker)
data = stock.earnings_dates
data

,EPS Estimate,Reported EPS,Surprise(%)
Earnings Date,,,
2025-02-05 05:00:00-05:00,NaN,NaN,NaN
2024-10-29 06:00:00-04:00,NaN,NaN,NaN
2024-07-30 06:00:00-04:00,NaN,NaN,NaN
2024-04-30 06:00:00-04:00,2.45,NaN,NaN
2024-02-07 16:00:00-05:00,1.78,1.93,0.0839
2023-10-31 16:00:00-04:00,1.61,1.77,0.1018
2023-08-01 16:00:00-04:00,1.59,1.62,0.0162
2023-05-02 16:00:00-04:00,2.35,2.46,0.0448
2023-02-07 16:00:00-05:00,1.49,1.73,0.1629


In [10]:
luist2 = []
for i in luist:
    print(i[0])
    luist2.append(i[0])


RE4.SI


In [12]:
stock = yf.Ticker('RE4.SI')
print(stock.earnings_dates)


                           EPS Estimate  Reported EPS  Surprise(%)
Earnings Date                                                     
2024-02-27 20:00:00+08:00           NaN           NaN          NaN
2023-11-14 19:00:00+08:00           NaN           NaN          NaN
2023-10-31 18:00:00+08:00           NaN           NaN          NaN
2023-08-10 18:00:00+08:00           NaN           NaN          NaN
2023-05-12 17:00:00+08:00           NaN           NaN          NaN
2023-02-27 17:00:00+08:00           NaN           NaN          NaN
2022-11-09 18:00:00+08:00           NaN           NaN          NaN
2022-08-11 17:00:00+08:00           NaN           NaN          NaN
2022-05-12 18:00:00+08:00           NaN           NaN          NaN
2022-02-24 17:00:00+08:00           NaN           NaN          NaN
2021-11-10 17:00:00+08:00           NaN           NaN          NaN
2021-08-11 15:00:00+08:00           NaN           NaN          NaN


In [104]:
pip install jupyterthemes

                                              0.0/7.0 MB ? eta -:--:--
     --                                       0.5/7.0 MB 10.7 MB/s eta 0:00:01
     ---                                      0.6/7.0 MB 6.4 MB/s eta 0:00:01
     -------                                  1.3/7.0 MB 9.3 MB/s eta 0:00:01
     ---------                                1.6/7.0 MB 11.5 MB/s eta 0:00:01
     ------------                             2.2/7.0 MB 9.8 MB/s eta 0:00:01
     -----------------                        3.0/7.0 MB 11.2 MB/s eta 0:00:01
     -----------------------                  4.1/7.0 MB 12.5 MB/s eta 0:00:01
     ---------------------------              4.8/7.0 MB 12.9 MB/s eta 0:00:01
     --------------------------------         5.7/7.0 MB 13.4 MB/s eta 0:00:01
     ------------------------------------     6.4/7.0 MB 13.6 MB/s eta 0:00:01
     ---------------------------------------  6.9/7.0 MB 13.3 MB/s eta 0:00:01
     ---------------------------------------- 7.0/7.0 MB 12.8 M

In [117]:
!jt -l

Available Themes: 
   chesterish
   grade3
   gruvboxd
   gruvboxl
   monokai
   oceans16
   onedork
   solarizedd
   solarizedl


In [1]:
!jt -t monokai